In [1]:
from pycaret.nlp import *
# import numpy as np
import pandas as pd
import sys
sys.path.append('/Users/nezmi/Projects/yeznable_projects')

from LAB.Data_collection import functions_for_db
from personal import id_pw_classes

In [2]:
# DB에 연결해놓기
# 로그인 정보들 숨길 필요가 있음
mysql_user = id_pw_classes.mysql()

host = '127.0.0.1'
user = mysql_user.id_yeznable
password = mysql_user.pw_yeznable
db = 'KAWS_instagram'

connection = functions_for_db.get_connection(host,user,password,db)
cursor = connection.cursor()
print('DB connected')

DB connected


In [3]:
# DB 커서를 생성해서 수집되어있는 댓글들과 ID를 받아옴
query_select_collected_postings_instances = "SELECT PostingID, PP_Posting FROM POSTINGS"
cursor.execute(query_select_collected_postings_instances)
tuple_collected_postings_instances = cursor.fetchall()      # ((PostingID, PP_Posting), ...)

In [4]:
dic_collected_postings_instances = {}
for item in tuple_collected_postings_instances:
    dic_collected_postings_instances[str(item[0])] = str(item[1])

# print(dic_collected_comments_instances)
dic_data = {'ID': list(dic_collected_postings_instances.keys()), 
            'Postings': list(dic_collected_postings_instances.values())}

In [5]:
data = pd.DataFrame(dic_data)
display(data)
print(data['Postings'].isnull().sum())

,ID,Postings
0,1,"peter saul, 1980 #post4thofjuly #petersaul"
1,2,serve @servefba
2,3,#saturday #kaws @acuteart_
3,4,"eugene von bruenchenhein, 1960 #eugenevonbruen..."
4,5,"revolt, rtw, mid 80’s @drrevoltrtw"
...,...,...
1639,1640,this is an ad i painted over and left on the s...
1640,1641,#tbt #nyc mostly 1997 #kaws
1641,1642,#repost @mrkimjones ・・・ summer vibes @kaws @di...
1642,1643,#goodbye #summer


0


In [6]:
nlp1 = setup(data, target = 'Postings', session_id = 123)

Description,Value
session_id,123
# Documents,1644
Vocab Size,2840
Custom Stopwords,False


In [7]:
# 8개의 카테고리를 가정
lda = create_model('lda', num_topics = 8)
print(lda)

LdaModel(num_terms=2840, num_topics=8, decay=0.5, chunksize=100)


In [8]:
plot_model(lda, plot = 'tsne')

In [9]:
plot_model(lda, plot = 'topic_distribution')

In [8]:
evaluate_model(lda)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Frequency Plot', 'freque…

In [12]:
lda_result = assign_model(lda)
display(lda_result)

,ID,Postings,Topic_0,Topic_1,Topic_2,Topic_3,Topic_4,Topic_5,Topic_6,Topic_7,Dominant_Topic,Perc_Dominant_Topic
0,1,,0.151648,0.067546,0.121391,0.211976,0.146200,0.095850,0.062282,0.143107,Topic 3,0.21
1,2,serve servefba,0.101285,0.045105,0.081071,0.307915,0.097639,0.064009,0.207401,0.095576,Topic 3,0.31
2,3,,0.151648,0.067546,0.121391,0.211976,0.146200,0.095850,0.062282,0.143107,Topic 3,0.21
3,4,,0.151648,0.067546,0.121391,0.211976,0.146200,0.095850,0.062282,0.143107,Topic 3,0.21
4,5,drrevoltrtw,0.163044,0.062771,0.118996,0.218989,0.143496,0.090558,0.057732,0.144414,Topic 3,0.22
...,...,...,...,...,...,...,...,...,...,...,...,...
1639,1640,ad paint leave day switch bus shelter ad frenc...,0.037396,0.401480,0.102244,0.084212,0.178095,0.009046,0.095417,0.092110,Topic 1,0.40
1640,1641,mostly,0.121406,0.054075,0.097182,0.369131,0.117044,0.076734,0.049861,0.114568,Topic 3,0.37
1641,1642,repost mrkimjone summer savethebee,0.075953,0.033830,0.060799,0.480516,0.073224,0.048006,0.031208,0.196464,Topic 3,0.48
1642,1643,summer,0.121400,0.054073,0.097179,0.169696,0.117039,0.076732,0.049859,0.314022,Topic 7,0.31
